# Modelling


In deze stap gaan we verschillende modellen toepassen op onze data en zo vergelijken om te kijken welk model het beste werkt op de data. 

Eerst herhalen we hieronder een aantal stappen die we al eerder hebben uitgevoerd. (Voor toelichting: zie 3 Data preparation)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

import datetime as date

from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import r2_score

n_jobs = 16 #set number of jobs equal or -1 the amount of cpu cores you have to speed up this notebook.

In [2]:
usefullcols = ['stm_mon_nr','stm_sap_meld_ddt','stm_status_melding_sap','stm_km_van_mld','stm_km_tot_mld','stm_km_van_gst',
'stm_km_tot_gst','stm_fh_tijd','stm_sap_melddatum','stm_aanngeb_tijd','stm_aanntpl_tijd','stm_arbeid',
'stm_progfh_in_tijd','stm_progfh_in_invoer_tijd','stm_progfh_in_duur','stm_sap_storeindtijd','stm_progfh_gw_tijd',
'stm_reactie_duur','stm_progfh_gw_duur','stm_progfh_gw_teller','stm_afspr_aanvangdd','stm_afspr_aanvangtijd',
'stm_fh_duur','stm_evb','stm_sap_meldtijd','stm_sap_meldtekst_lang','stm_prioriteit','stm_oh_pg_gst',
'stm_sap_meldtekst','stm_techn_gst','stm_contractgeb_gst','stm_tao_indicator','stm_geo_mld','stm_functiepl_mld',
'stm_geo_mld_uit_functiepl','stm_aanngeb_ddt','stm_aanngeb_dd','stm_oorz_code','stm_oorz_groep','stm_oorz_tkst',
'stm_fh_dd','stm_fh_status','stm_geo_gst','stm_functiepl_gst','stm_geo_gst_uit_functiepl','stm_fh_ddt',
'stm_aanntpl_dd','stm_techn_mld','stm_sap_storeinddatum','stm_equipm_nr_mld','stm_equipm_soort_mld',
'stm_equipm_omschr_mld','stm_sap_storeind_ddt','stm_contractgeb_mld','stm_equipm_nr_gst','stm_equipm_soort_gst',
'stm_equipm_omschr_gst','stm_progfh_in_invoer_dat','stm_progfh_in_datum','stm_oorz_tekst_kort','stm_dstrglp_naar',
'stm_tao_indicator_vorige','stm_vl_post','stm_dstrglp_van','stm_pplg_van','stm_tao_soort_mutatie',
'stm_progfh_gw_lwd_tijd','stm_pplg_naar','stm_progfh_gw_lwd_datum']

In [3]:
df = pd.read_csv('sap_storing_data_hu_project.csv',usecols=usefullcols, low_memory=True)
# df = pd.read_csv('sap_storing.csv',usecols=usefullcols, low_memory=True)

/Users/jet/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11,24,27,49,51,55,56,73,75,78,80,82,86,87,91,95,123,124) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
features = ['stm_prioriteit','stm_oorz_code','stm_oorz_groep','stm_equipm_soort_*','stm_geo_*']
target = 'duur'

In [5]:
now = date.datetime.today()
zeroTimeDiff = now - now

df.drop_duplicates(subset=['stm_mon_nr'], keep='last',inplace = True)

df['stm_aanntpl_ddt'] = pd.to_datetime(df['stm_aanntpl_dd']+' '+df['stm_aanntpl_tijd'], format='%d/%m/%Y %H:%M:%S')
df['stm_fh_ddt'] = pd.to_datetime(df['stm_fh_ddt'], format='%d/%m/%Y %H:%M:%S')

df.dropna(subset=['stm_aanntpl_dd','stm_aanntpl_tijd','stm_fh_ddt'],inplace=True)

df['duur'] = df['stm_fh_ddt'] - df['stm_aanntpl_ddt']
df.dropna(subset=['duur'],inplace=True)

# Meldingen waar de oplossing in de toekomst is zijn onzin.
# df[df['stm_sap_storeind_ddt'] >= now][['stm_sap_storeind_ddt','stm_sap_meld_ddt']]
# df.drop(df[df['stm_sap_storeind_ddt'] >= now][['stm_sap_storeind_ddt','stm_sap_meld_ddt']].index,inplace=True)

# Rijen waar het probleem sneller is opgelost dan 5 minuten (en ook de meldingen tijd) zijn onbetrouwbaar.
df.drop(df[df['duur'] <= date.timedelta(minutes=5)].index, inplace = True)

# # Alles wat langer dan een dag duurt is niet urgent genoeg en dus niet nuttig.
df.drop(df[df['duur'] > date.timedelta(hours=6)].index, inplace = True)

In [6]:
dubbleCol = ['stm_equipm_omschr_*', 'stm_equipm_soort_*', 'stm_equipm_nr_*', 
             'stm_geo_*_uit_functiepl', 'stm_functiepl_*', 'stm_geo_*',
            'stm_km_van_*','stm_km_tot_*',
            'stm_contractgeb_*','stm_techn_*']
original = 'mld'
optional = ['gst'] # Order of least to most important
for colPH in dubbleCol:
    colOg = colPH.replace('*',original)
    
    for option in optional:
        colOp = colPH.replace('*',option)
        df[colPH] = np.where(df[colOp].isna(), df[colOg], df[colOp])
        df.drop(columns=[colOp],inplace=True)
    df.drop(columns=[colOg],inplace=True)

In [7]:
df['stm_prioriteit'] = df['stm_prioriteit'].fillna(1) # Laagste prio
df['stm_oorz_code'] = df['stm_oorz_code'].fillna(299) # 'Niet gemeld' code
df['stm_oorz_groep'] = df['stm_oorz_groep'].fillna('x')
df['stm_fh_status'] = df['stm_fh_status'].fillna(1) # Median of values
df['stm_aanngeb_tijd'] = df['stm_aanngeb_tijd'].fillna(df['stm_sap_meldtijd']) # Aannemer beltijd gelijk met melding zetten.
df['stm_progfh_in_invoer_tijd'] = df['stm_progfh_in_invoer_tijd'].fillna(df['stm_aanngeb_tijd']) # Tijd van prognose op beltijd zetten.
df['stm_equipm_nr_*'] = df['stm_equipm_nr_*'].fillna(0)
df['stm_equipm_soort_*'] = df['stm_equipm_soort_*'].fillna('x')
df['stm_contractgeb_*'] = df['stm_contractgeb_*'].fillna(0)
df['stm_progfh_gw_teller'] = df['stm_progfh_gw_teller'].fillna(0)

df['stm_prioriteit'] = df['stm_prioriteit'].astype(int)
df['stm_oorz_code'] = df['stm_oorz_code'].astype(int)
df['stm_fh_status'] = df['stm_fh_status'].astype(int)
df['stm_fh_duur'] = df['stm_fh_duur'].astype(int)
df['stm_progfh_gw_teller'] = df['stm_progfh_gw_teller'].astype(int)
df['stm_equipm_nr_*'] = df['stm_equipm_nr_*'].astype(int)
df['stm_contractgeb_*'] = df['stm_contractgeb_*'].astype(int)

df['duur'] = df['duur'].apply(lambda x: x.seconds / 60)
df['duur'] = df['duur'].astype(int)

## 1. Baseline model 

We beginnen met het baseline model. Dit is het meest simpele model waarvan de uitkomst is dat alle storingen in 2 uur en 17 minuten opgelost zijn.

### Score en RMSE
Echter kunnen we aan de score en de RMSE aflezen dat het baseline model niet erg geschikt is. 

In [8]:
amntInTime = df[df['duur'] < date.timedelta(hours=2,minutes=17).seconds / 60].count()[0]
total = df.shape[0]
baseline = date.timedelta(hours=2,minutes=17).seconds / 60
pred = [int(baseline)]*df[['duur']].shape[0]

baseline = date.timedelta(hours=2,minutes=17)
baseline = baseline.seconds / 60
pred = [int(baseline)]*df[[target]].shape[0]

print('Score :', r2_score(df[[target]],pred))
print('Root mean squared error:',np.sqrt(mean_squared_error(df['duur'],pred)))

Score : -1.0368133962977053
Root mean squared error: 95.1771464162169


## 2. Dataset trainen

In [9]:
le_ozgr = LabelEncoder()
le_ozgr.fit(df['stm_oorz_groep'].unique())
df['stm_oorz_groep'] = le_ozgr.transform(df['stm_oorz_groep'])
# df['stm_oorz_groep'].unique()
pickle.dump(le_ozgr, open('le_ozgr', 'wb'))

In [10]:
le_equipsrt = LabelEncoder()
le_equipsrt.fit(df['stm_equipm_soort_*'].unique())
df['stm_equipm_soort_*'] = le_equipsrt.transform(df['stm_equipm_soort_*'])
# df['stm_equipm_soort_*'].unique()
pickle.dump(le_equipsrt, open('le_equipsrt', 'wb'))

###  2.1  Get_dummies
Hieronder gebruiken we get_dummies op verschillende feature kolommen, wat we hier doen is het omzetten van categoriale data naar nominale waardes. We doen dit niet bij stm_prioriteit omdat het voor deze kolom belangrijk is dat het zijn volgorde behoudt, de waarde van de prioriteit zegt iets over de volgordelijkheid. Wanneer je hierop get_dummies zou toepassen, zou je juist deze volgordelijkheid verliezen. Bij de andere kolommen is dit niet zo. 

In [11]:
dOC = pd.get_dummies(df['stm_oorz_code'],prefix='oorz_code')
dOG = pd.get_dummies(df['stm_oorz_groep'],prefix='oorz_groep')
eqs = pd.get_dummies(df['stm_equipm_soort_*'])
geo = pd.get_dummies(df['stm_geo_*'],prefix='geo')

dummies = pd.concat([df['stm_prioriteit'],dOC, dOG, eqs, geo],axis=1)

display(dummies)
dummies.shape

,stm_prioriteit,oorz_code_33,oorz_code_130,oorz_code_131,oorz_code_132,oorz_code_133,oorz_code_134,oorz_code_135,oorz_code_136,oorz_code_140,...,geo_924,geo_924.0,geo_926,geo_94.0,geo_950,geo_950.0,geo_952,geo_98.0,geo_99,geo_99.0
32099,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60683,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85621,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91875,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92852,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898355,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
898368,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
898413,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
898475,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


(66657, 975)

In [12]:
 X = dummies
#X = df[features]
Y = df[target]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=21)

In [13]:
print(y_train,y_train.shape)

166704     40
469546     52
752187     60
246706     19
596457     32
         ... 
817163    210
492952     17
463751     33
810326     25
235216     10
Name: duur, Length: 59991, dtype: int64 (59991,)


## 2. Lineaire regressie

### Score en RMSE

In [14]:
linearModel = linear_model.LinearRegression().fit(x_train,y_train)
linYPrediction = linearModel.predict(x_test)
print(f'Score: {linearModel.score(x_test,y_test)}')
print(f'Mean squared error: {np.sqrt(mean_squared_error(y_test,linYPrediction))}')

Score: -1.512165349501866e+16
Mean squared error: 8209374840.113075


In [15]:
display(x_test[:5])
display(y_test[:5])
linYPrediction[:5]

,stm_prioriteit,oorz_code_33,oorz_code_130,oorz_code_131,oorz_code_132,oorz_code_133,oorz_code_134,oorz_code_135,oorz_code_136,oorz_code_140,...,geo_924,geo_924.0,geo_926,geo_94.0,geo_950,geo_950.0,geo_952,geo_98.0,geo_99,geo_99.0
436311,2,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241351,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
666482,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
524448,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372190,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


436311    103
241351    126
666482     60
524448      8
372190    294
Name: duur, dtype: int64

array([62.47851562, 87.92382812, 89.05273438, 37.40429688, 46.82226562])

## 3. Ridge regressie

### Score en RMSE

In [16]:
ridgeModel = linear_model.Ridge(alpha=1).fit(x_train,y_train)
ridgeYPrediction = ridgeModel.predict(x_test)
print(f'Score: {ridgeModel.score(x_test,y_test)}')
print(f'Mean squared error: {np.sqrt(mean_squared_error(y_test,ridgeYPrediction))}')
print(ridgeModel.get_params())
print(ridgeModel.get_params(deep=True))

Score: 0.1285415379485525
Mean squared error: 62.32091062744355
{'alpha': 1, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'tol': 0.001}
{'alpha': 1, 'copy_X': True, 'fit_intercept': True, 'max_iter': None, 'normalize': False, 'random_state': None, 'solver': 'auto', 'tol': 0.001}


## 4. KNearest Neighbours regressie
### Score en RMSE

In [17]:
knearestModel = KNeighborsRegressor(n_neighbors=2,n_jobs=n_jobs).fit(x_train,y_train)
knearestYPrediction = knearestModel.predict(x_test)
print(f'Score: {knearestModel.score(x_test,y_test)}')
print(f'Mean squared error: {np.sqrt(mean_squared_error(y_test,knearestYPrediction))}')

Score: -0.18095928719327903
Mean squared error: 72.54841152431574


### Bias/Variance trade-off

In [18]:
# max_neighbours = 10
# scores_train = np.zeros(max_neighbours)
# scores_test = np.zeros(max_neighbours)
# variance = np.zeros(max_neighbours)

# neighbours = range(1, max_neighbours + 1)
# for n in neighbours:
#     print(f"Starting with run:{n}")
#     model = KNeighborsRegressor(n_neighbors=n,n_jobs=n_jobs,leaf_size=10)
#     model.fit(x_train, y_train)
#     score_train = model.score(x_train, y_train)
#     scores_train[n-1] = score_train

#     score_test = model.score(x_test, y_test)
#     scores_test[n-1] = score_test

#     variance[n-1] = score_train - score_test
#     #     print("Score bij diepte {}: {:.2f}%.".format(depth, score))

# fig, ax = plt.subplots(1, dpi=100)

# line1, = ax.plot(neighbours, scores_train, label='Train')
# line2, = ax.plot(neighbours, scores_test, label='Test')
# line3, = ax.plot(neighbours, variance, label='Variance')

# ax.legend()
# ax.set_xlabel("diepte (aantal lagen)")
# ax.set_ylabel("Bias-Variance")

# plt.show()

## 5. Decision Tree regressie

In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score

Bij decision tree regressie kun je zelf de parameter voor max_depth aanpassen. Hieronder tonen we twee voorbeelden waarbij bij het eerste geval de max_depth op 2 staat en bij het tweede geval op 37. Hierna vergelijken we de scores van de twee decision trees. 

In [20]:
dtree1 = DecisionTreeRegressor(max_depth=2)
dtree2 = DecisionTreeRegressor(max_depth=37)
dtree1.fit(x_train, y_train)
dtree2.fit(x_train, y_train)

# Code Lines 5 to 6: Predict on training data
tr1 = dtree1.predict(x_train)
tr2 = dtree2.predict(x_train) 

#Code Lines 7 to 8: Predict on testing data
y1 = dtree1.predict(x_test)
y2 = dtree2.predict(x_test) 

### Score en RMSE

#### Test decision trees

In [21]:
print("Score decision tree 1, max_depth 2")
print(np.sqrt(mean_squared_error(y_test,y1))) 
print(r2_score(y_test, y1))

Score decision tree 1, max_depth 2
65.54789427157387
0.03595657000510666


In [22]:
print("Score decision tree 2, max_depth 37")
print(np.sqrt(mean_squared_error(y_test,y2))) 
print(r2_score(y_test, y2))

Score decision tree 2, max_depth 37
66.32782565533752
0.01287845410756916


#### Final decision tree

Om overfitten te voorkomen bij een te hoge max_depth en wel de beste score te behouden, kiezen we voor deze hyperparamter het getal 8. 

In [23]:
dtree_final = DecisionTreeRegressor(max_depth=8)
dtree_final.fit(x_train, y_train)

y_final = dtree_final.predict(x_test)

print(r2_score(y_test, y_final))
print(np.sqrt(mean_squared_error(y_test,y_final))) 

0.08965249279376963
63.69627695312084


### Bias/Variance trade-off

In [24]:
# max_depth = 20
# scores_train = np.zeros(max_depth)
# scores_test = np.zeros(max_depth)
# variance = np.zeros(max_depth)

# depths = range(1, max_depth + 1)
# for depth in depths:
#     model = DecisionTreeRegressor(max_depth=depth, min_samples_split=20)
#     model.fit(x_train, y_train)
    
#     score_train = model.score(x_train, y_train)
#     scores_train[depth-1] = score_train

#     score_test = model.score(x_test, y_test)
#     scores_test[depth-1] = score_test

#     variance[depth-1] = score_train - score_test
#     #     print("Score bij diepte {}: {:.2f}%.".format(depth, score))

# fig, ax = plt.subplots(1, dpi=100)

# line1, = ax.plot(depths, scores_train, label='Train')
# line2, = ax.plot(depths, scores_test, label='Test')
# line3, = ax.plot(depths, variance, label='Variance')

# ax.legend()
# ax.set_xlabel("diepte (aantal lagen)")
# ax.set_ylabel("Bias-Variance")

# plt.show()

## 6. Random Forest regressie

In [25]:
model_rf = RandomForestRegressor(max_features=None, n_estimators=50, max_leaf_nodes=48, oob_score=True, random_state=100,n_jobs=n_jobs)

### Score en RMSE

In [26]:
model_rf.fit(x_train, y_train) 
pred_train_rf= model_rf.predict(x_train)
print('Train RMSE:',np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print('Train R^2 score:',r2_score(y_train, pred_train_rf))
pred_test_rf= model_rf.predict(x_test)
print('Test RMSE:',np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print('Test R^2 score:',r2_score(y_test, pred_test_rf))

Train RMSE: 62.25818256850878
Train R^2 score: 0.12826947739556394
Test RMSE: 62.29861607909364
Test R^2 score: 0.12916493376924598


Aangezien dit model het best scoort wat betreft de r2 score en de RMSE, kiezen we dit model om de voorspellingen mee te gaan doen. 

### 6.1 Probability functie

We willen graag weten hoe zeker het model is van een voorspelling: hiervoor gebruiken we de probability functie. 

In [27]:
model = RandomForestRegressor(
    max_features=None, n_estimators=50, max_leaf_nodes=48, oob_score=True, random_state=100,n_jobs=n_jobs
                             ).fit(x_train, y_train)

In [28]:
def random_forest_regressor_predict_proba(X_train, y_train, X_test, m):
    """Trains DecisionTreeRegressor model and predicts probabilities of each y.

    Args:
        X_train: Training features.
        y_train: Training labels.
        X_test: New data to predict on.
        **kwargs: Other arguments passed to DecisionTreeRegressor.

    Returns:
        DataFrame with columns for record_id (row of X_test), y 
        (predicted value), and prob (of that y value).
        The sum of prob equals 1 for each record_id.
    """
    # Get y values corresponding to each node.
    node_ys = pd.DataFrame({'node_id':m.apply(X_train)[:,0], 'duur': y_train})
    # Calculate probability as 1 / number of y values per node.
    node_ys['prob'] = 1 / node_ys.groupby(node_ys.node_id).transform('count')
    # Aggregate per node-y, in case of multiple training records with the same y.
    node_ys_dedup = node_ys.groupby(['node_id', 'duur']).prob.sum().to_frame()\
        .reset_index()
    display(node_ys_dedup)
#     node_ys_dedup.to_csv('nodes_prob.csv')
    
    dept = m.decision_path(X_test)[1][1]
    leaf = pd.DataFrame(m.decision_path(X_test)[0].toarray()[:,:dept]).apply(
        lambda x:x.to_numpy().nonzero()[0].max(), axis=1).to_frame(
            name='node_id')
    leaf['record_id'] = leaf.index
    # Merge with y values and drop node_id.
    return leaf.merge(node_ys_dedup, on='node_id').drop(
        'node_id', axis=1).sort_values(['record_id', 'duur'])

In [29]:
res = random_forest_regressor_predict_proba(x_train, y_train, x_test[:1],model)

,node_id,duur,prob
0,6,6,0.007792
1,6,7,0.007792
2,6,8,0.005195
3,6,9,0.002597
4,6,10,0.007792
...,...,...,...
6662,93,348,0.003436
6663,93,349,0.003436
6664,94,63,0.333333
6665,94,280,0.333333


In [30]:
res[res['record_id'] == 0].sort_values('prob',ascending=False)[['duur','prob']]

,duur,prob
25,30,0.022603
10,15,0.020653
5,10,0.019810
20,25,0.017492
15,20,0.017176
...,...,...
341,351,0.000053
297,302,0.000053
343,353,0.000053
326,334,0.000053


In [31]:
# filename = 'finalized_model_lin.sav'
# pickle.dump(dtree_final, open(filename, 'wb'))
# filename = 'finalized_model_dectreereg.sav'
# pickle.dump(dtree_final, open(filename, 'wb'))
# filename = 'finalized_model_knearest.sav'
# pickle.dump(knearestModel, open(filename, 'wb'))
# df.to_csv('sap_storing_data_hu_project_norm.csv', columns=features)

filename = 'rand_for.sav'
pickle.dump(model_rf, open(filename, 'wb'))